In [1]:
from utilities import init_logger
from queryLabel import Taxonomy

In [2]:
# Inputs
CATEGORIES_PATH = '/scratch/WikipediaImagesTaxonomy/commonswiki-20220220-category-network.parquet'
GRAPH_PATH = '/scratch/WikipediaImagesTaxonomy/20220220-category-graph.pkl.bz2'
FILES_PATH = '/scratch/WikipediaImagesTaxonomy/commonswiki-20220220-files.parquet'

In [3]:
logger = init_logger('category.log')

In [4]:
taxonomy = Taxonomy()

In [5]:
init = 0
if(init):
    taxonomy.load_categories(CATEGORIES_PATH)
    print('Dumping..')
    taxonomy.dump_graph(GRAPH_PATH)

In [6]:
taxonomy.load_graph(GRAPH_PATH)

In [7]:
taxonomy.set_taxonomy(mapping='content_extended')

In [8]:
taxonomy.get_head('CommonsRoot')

2022-05-08 23:11:23 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-05-08 23:11:23 INFO: Use device: gpu
2022-05-08 23:11:23 INFO: Loading: tokenize
2022-05-08 23:11:26 INFO: Loading: pos
2022-05-08 23:11:41 INFO: Loading: constituency
2022-05-08 23:11:42 INFO: Done loading processors!


'Commonsroot'

In [ ]:
taxonomy.get_label('Comedy films of the United States')

In [52]:
taxonomy.get_label('Balconies in Sicily')

{'Nature'}

In [53]:
taxonomy.get_label('2011 Ukrainian Superleague All-Star game')

{'Events'}

In [58]:
taxonomy.get_label('11th-century bookbinding')

set()